In [1]:
import requests
import pandas as pd
import re
import time
from bs4 import BeautifulSoup as bs
from IPython.core.debugger import set_trace

In [2]:
class Researcher():
    def __init__(self,link,source):
        self.link = link
        self.request = requests.get(link)
        self.source = source
        self.request.encoding='UTF-8'
        self.soup = bs(self.request.text,'lxml')
        self.name = self.parseH2()
        self.characteristics = self.parseCharacteristics()
        self.courses = self.parseCourses()
        self.theses = self.parseTheses()
        
    def parseCharacteristics(self):
        nextSiblings = ['Akademická funkce:','Odborné zaměření:','Členství:','Kancelář:','Email:','Telefon:','Konzultační hodiny:']       
        ser_nextSiblings = self.parseNextSiblings(nextSiblings)
        
        parentsNextSiblings = ['Člen organizací','Vzdělání','Odborná praxe','Veřejné aktivity']
        ser_parentsNextSiblings = self.parseParentsNextSibling(parentsNextSiblings)
        
        chars = pd.concat([ser_nextSiblings,ser_parentsNextSiblings])
        chars.loc['researcher'] = self.name
        chars.loc['source'] = self.source
        return chars
        
    def parseH2(self):
        return self.soup.find('h2').text.strip()
    
    def parseNextSiblings(self,attrs):
        d = {}
        for attr in attrs:
            el = self.soup.find('strong',text=attr)
            if el:
                text = el.next_sibling
                if text is not None:
                    d[attr] = text.strip()
                else:
                    d[attr] = None
        return pd.Series(d)
        
    def parseParentsNextSibling(self,attrs):
        d = {}
        for key in attrs:
            el = self.soup.find('strong',text=key)
            if el:
                d[key] = el.parent.next_sibling
            
        return pd.Series(d)
    
    def parseCourses(self):
        courses_html = ''
        h3s = self.soup.find("h3",text='Předměty')
        if h3s:
            for tag in h3s.next_siblings:
                if tag.name == "h3":
                    break
                else:
                    courses_html += str(tag)
            courses_soup = bs(courses_html,'lxml')
            courses = courses_soup.select('a[href*=syllab]')
            roles = [course.parent.find_previous_sibling('h4').text for course in courses]
            names = [course.text.split(' - ')[1].strip() for course in courses]
            researchers = [self.name] * len(courses)
            idents = [course.text.split(' - ')[0].strip() for course in courses]

            return pd.DataFrame({'role':roles,'name':names,'researcher':researchers,'ident':idents,'source':[self.source] * len(courses)})
        else:
            return pd.DataFrame({'role':[],'name':[],'researcher':[],'ident':[],'source':[]})
    
    def parseTheses(self):
        d = {'bakalářské':'Vedoucí bakalářských prací','diplomové':'Vedoucí diplomových prací'}
        theses = pd.DataFrame()
        for key in d:
            el = self.soup.find('h3',text=d[key])
            if el:
                counts = el.next_sibling.next_sibling.find('strong').next_sibling.strip()
                total = int(counts.split('/')[0])
                marked = int(counts.split('/')[1])
                self.characteristics.loc[key + '_vše'] = total
                self.characteristics.loc[key + '_oceněné'] = marked
                
                if marked > 0:
                    works = el.next_sibling.next_sibling.select('a[href*=work]')
                    names = [work.text for work in works]
                    links = [work['href'] for work in works]
                    bcmgr = [key] * len(works)

                    df = pd.DataFrame({'name':names,'link':links,'type':bcmgr,'researcher':[self.name] * len(works),'source':[self.source]*len(works)})
                    theses = pd.concat([theses,df])
        return theses

In [3]:
start_links = {'Interní':'http://ies.fsv.cuni.cz/cs/node/48','Externí':'http://ies.fsv.cuni.cz/cs/node/49','Doktorandi':'http://ies.fsv.cuni.cz/cs/node/51'}

researchers = []
lcourses = []
ltheses = []
lchars = []

for key in start_links:
    r = requests.get(start_links[key])
    r.encoding = 'UTF-8'
    soup = bs(r.text,'lxml')
    tds = soup.find_all('td',{'class':'peopleTableCellName'})
    links = ['http://ies.fsv.cuni.cz/' + td.a['href'] for td in tds]

    i=0
    for link in links:
        res = Researcher(link,key)
        researchers.append(res)
        lcourses.append(res.courses)
        ltheses.append(res.theses)
        lchars.append(res.characteristics)
        print('Success: {},({})'.format(res.name,i))
        i += 1
        time.sleep(0.5)

courses = pd.concat(lcourses)
theses = pd.concat(ltheses)
characteristics = pd.concat(lchars,axis=1).T
characteristics = characteristics.set_index('researcher')

Success: doc. PhDr. Jozef Baruník Ph.D.,(0)
Success: doc. PhDr. Michal Bauer Ph.D.,(1)
Success: PhDr. Jaromír Baxa Ph.D.,(2)
Success: PhDr. Lucie Bryndová,(3)
Success: doc. Ing. Tomáš Cahlík CSc.,(4)
Success: PhDr. František Čech,(5)
Success: RNDr. Michal Červinka Ph.D.,(6)
Success: doc. PhDr. Julie Chytilová Ph.D.,(7)
Success: prof. Ing. Oldřich Dědek CSc.,(8)
Success: doc. PhDr. Ing. Antonie Doležalová Ph.D.,(9)
Success: doc. PhDr. Adam Geršl Ph.D.,(10)
Success: doc. PhDr. Martin Gregor Ph.D.,(11)
Success: doc. PhDr. Tomáš Havránek Ph.D.,(12)
Success: doc. PhDr. Zuzana Havránková Ph.D.,(13)
Success: PhDr. Michal Hlaváček Ph.D.,(14)
Success: Ing. Monika Hollmannová,(15)
Success: doc. Mgr. Tomáš Holub Ph.D.,(16)
Success: prof. Roman Horváth Ph.D.,(17)
Success: doc. PhDr. Ing. Ing. Petr Jakubík Ph.D. Ph.D.,(18)
Success: prof. Ing. Karel Janda M.A., Dr., Ph.D.,(19)
Success: doc. Petr Janský Ph.D.,(20)
Success: Ing. Irena Kemény,(21)
Success: prof. Ing. Evžen Kočenda M.A., Ph.D., DSc.,(22

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [4]:
characteristics

,Akademická funkce:,Email:,Kancelář:,Konzultační hodiny:,Odborná praxe,Odborné zaměření:,Telefon:,Veřejné aktivity,Vzdělání,bakalářské_oceněné,bakalářské_vše,diplomové_oceněné,diplomové_vše,source,Člen organizací,Členství:
researcher,,,,,,,,,,,,,,,,
doc. PhDr. Jozef Baruník Ph.D.,Associate Professor,barunik [AT] fsv [DOT] cuni [DOT] cz,503,dle dohody,<p>2017 - research visit Humboldt-Universität ...,"Finanční ekonomie, ekonometrie",+420(776)259273,<p>2018+ President of the Czech Econometric So...,"<p>2011 PhD ekonomie, IES FSV UK<br/> 2006 Ph...",0,4,20,54,Interní,"<p>The Econometric Society, The Society for Fi...","Interní, Katedra makroekonomie a ekonometrie"
doc. PhDr. Michal Bauer Ph.D.,Associate Professor,bauerm [AT] fsv [DOT] cuni [DOT] cz,402,kontaktujte me prosim predem emailem,<p>ACADEMIC AFFILIATIONS<br/> <br/> 2017+ CE...,"Hospodářský rozvoj chudých zemí, rozhodování j...",222 112 329,NaN,NaN,1,5,5,9,Interní,NaN,"Interní, Katedra institucionální ekonomie"
PhDr. Jaromír Baxa Ph.D.,Assistant Professor,jaromir [DOT] baxa [AT] fsv [DOT] cuni [DOT] cz,311,by appointment,<p>2006 - vyučující na IES FSV UK: magisterské...,"Makroekonomie, institucionální ekonomie",222 112 309,<p>2012 - Liberecký kraj (zastupitel)<br/> 20...,"<p>2006-2013 PhD student, IES FSV UK<br/> 200...",2,8,7,38,Interní,NaN,"Interní, Katedra makroekonomie a ekonometrie"
PhDr. Lucie Bryndová,"Adjunct Lecturer, Research Fellow",lucie [DOT] bryndova [AT] centrum [DOT] cz,,po dohodě via email,"<p>2008+ JEM101 Health Economics, IES FSV UK <...",Ekonomie zdravotnictví,,NaN,"<p>2004 - 05 doktorský vědecký program, CEFRES...",1,2,1,3,Interní,NaN,"Interní, Katedra evropské ekonomické integrace..."
doc. Ing. Tomáš Cahlík CSc.,Associate Professor,cahlik [AT] fsv [DOT] cuni [DOT] cz,410,čtvrtek 16.00 - 17.00 místnost č. 410,<p>1980-85 Výuka Operačního výzkumu a ekonomet...,"Makroekonomie, hospodářský rozvoj a růst, komp...",222 112 318,<p>2000 + Člen České společnosti ekonomické (...,<p>1975-79 Matematické metody v ekonomii na VŠ...,3,60,8,39,Interní,NaN,"Interní, Katedra makroekonomie a ekonometrie"
PhDr. František Čech,Junior Researcher,frantisek [DOT] cech [AT] fsv [DOT] cuni [DOT] cz,503,po dohode,"<p>2016+: <a href=""http://gemclime.cuni.cz/"">G...",Financial Econometrics; Time-Series Models; Po...,+420 776 535 106,<p>Výuka:<br/> ZS 2018/2019: JEB120 - Financi...,<p>2018: short research visit at Department of...,NaN,NaN,0,3,Interní,"<p><a href=""http://dyme.vse.cz/"">DYME – Dynami...","Doktorandi, Interní, Katedra financí a kapitál..."
RNDr. Michal Červinka Ph.D.,Senior Lecturer,michal [DOT] cervinka [AT] fsv [DOT] cuni [DOT...,"O408(IES), 225(UTIA)",Tuesdays 1pm-2.30pm; office 408 in Opletalova....,<p>Department of Decision Making Theory <br/> ...,"Oligopolní trhy, Nashova rovnováha, variační a...",+420 26605 2345,<p>člen Studijní komise AS FSV UK<br/> <br/> ...,<p>2003 - 2008: Faculty of Mathematics and Phy...,4,15,1,2,Interní,<p>Czech Society for Operations Research Ce...,"Interní, Katedra makroekonomie a ekonometrie"
doc. PhDr. Julie Chytilová Ph.D.,"Associate Professor, Head of Department of Ins...",chytilova [AT] fsv [DOT] cuni [DOT] cz,410,Úterý 9:30-10:50; konzultační hodiny zrušeny 2...,"<p>ZAMĚSTNÁNÍ<br/> 2016+ docentka, Institut e...","Behaviorální ekonomie, experimentální ekonomie...",222 112 318,NaN,<p>2004-2009 doktorandské studium IES FSV UK<b...,5,27,2,5,Interní,NaN,"Interní, Katedra institucionální ekonomie"
prof. Ing. Oldřich Dědek CSc.,"Professor, Head of Department of Macroeconomics",dedek [AT] fsv [DOT] cuni [DOT] cz,510,po dohodě e-mailem,<p>1978 - 1992 vědecký pracovník se zaměřením ...,"Měnová politika, evropská integrace",222 112 325,<p>1992 - 1996 náměstek ředitele Institutu eko...,"<p>květen 1978: titul inženýr, Vysoká škola ek...",3,19,8,43,Interní,NaN,"Interní, Katedra makroekonomie a ekonometrie"


In [5]:
courses

,role,name,researcher,ident,source
0,Garant,Advanced Econometrics,doc. PhDr. Jozef Baruník Ph.D.,JEM005,Interní
1,Garant,Kvantitativní metody I,doc. PhDr. Jozef Baruník Ph.D.,JED414,Interní
2,Garant,Kvantitativní metody II,doc. PhDr. Jozef Baruník Ph.D.,JED415,Interní
3,Garant,Nonlinear Dynamic Economic Systems: Theory and...,doc. PhDr. Jozef Baruník Ph.D.,"JED412,413",Interní
4,Garant,Quantitative Finance I,doc. PhDr. Jozef Baruník Ph.D.,JEM059,Interní
5,Garant,Quantitative Finance II,doc. PhDr. Jozef Baruník Ph.D.,JEM061,Interní
6,Vyučující,Advanced Econometrics,doc. PhDr. Jozef Baruník Ph.D.,JEM005,Interní
7,Vyučující,Applied Econometrics,doc. PhDr. Jozef Baruník Ph.D.,JEM116,Interní
8,Vyučující,Kvantitativní metody I,doc. PhDr. Jozef Baruník Ph.D.,JED414,Interní
9,Vyučující,Kvantitativní metody II,doc. PhDr. Jozef Baruník Ph.D.,JED415,Interní


In [6]:
theses

,name,link,type,researcher,source
0,Mgr. Adam Nedvěd,/work/index/show/id/2780/lang/cs,diplomové,doc. PhDr. Jozef Baruník Ph.D.,Interní
1,Mgr. Daniel Benčík,/work/index/show/id/1638/lang/cs,diplomové,doc. PhDr. Jozef Baruník Ph.D.,Interní
2,Mgr. Diana Burdová,/work/index/show/id/1951/lang/cs,diplomové,doc. PhDr. Jozef Baruník Ph.D.,Interní
3,Mgr. Eva Brabcová,/work/index/show/id/1310/lang/cs,diplomové,doc. PhDr. Jozef Baruník Ph.D.,Interní
4,Mgr. František Čech,/work/index/show/id/1757/lang/cs,diplomové,doc. PhDr. Jozef Baruník Ph.D.,Interní
5,Mgr. Jana Mašková,/work/index/show/id/1460/lang/cs,diplomové,doc. PhDr. Jozef Baruník Ph.D.,Interní
6,Mgr. Jiří Kukačka,/work/index/show/id/1491/lang/cs,diplomové,doc. PhDr. Jozef Baruník Ph.D.,Interní
7,Mgr. Jiří Poláček,/work/index/show/id/2105/lang/cs,diplomové,doc. PhDr. Jozef Baruník Ph.D.,Interní
8,Mgr. Ladislav Krištoufek,/work/index/show/id/1112/lang/cs,diplomové,doc. PhDr. Jozef Baruník Ph.D.,Interní
9,Mgr. Lucie Kraicová,/work/index/show/id/1897/lang/cs,diplomové,doc. PhDr. Jozef Baruník Ph.D.,Interní
